In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py
from tqdm import tqdm

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

In [5]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, DataManager_Class

In [6]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import * # import NumericalFunctions 

In [7]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [8]:
####################################
#LOADING CLASSES

In [17]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str, #just need this one for the data loading function
                                ModelData.Np_str, dataType="Animations", dataName="Animations",
                                dtype="float32", make_dirs=False)

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/Animations
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Proj

In [18]:
####################################
#FUNCTIONS

In [19]:
#TIME 00:00:00 Function
#Gets the realtime for the current timestep
def get_time(t):
    # init_day,init_hour,init_min=0,0,0
    init_day,init_hour,init_min=0,6,0
    times=ModelData.time/(1e9 * 60); time_inc=times.astype(int)[1]-times.astype(int)[0]
    current_min=init_hour*60+init_min+time_inc*t;
    
    days = init_day + (current_min // (24 * 60))
    
    remain_min = (init_min+time_inc*t) % (24 * 60); 
    hours = (init_hour + (remain_min // 60)) % 24
    mins = remain_min % 60

    ##############################################
    days=str(days);hours=str(hours);mins=str(mins)
    if len(days)==1:days='0'+days
    if len(hours)==1:hours='0'+hours
    if len(mins)==1:mins='0'+mins
    ##############################################

    combo=days+":"+hours+":"+mins
    return(days,hours,mins),(combo)

In [27]:
def minmax_surface_val_2D(data, var):
    #Data must be dimension (Time,Y,X)
    print(f'working on consistent contour value for {var}')
    data_var = data[var]
    
    t=0
    min_val=data_var.isel(time=t).min(dim=['yh', 'xh']).item()
    max_val=data_var.isel(time=t).max(dim=['yh', 'xh']).item()
    
    for t in np.arange(1,len(ModelData.time)):
        if np.mod(t,50)==0: print(f'current time {t}')
    
        current_min=data_var.isel(time=t).min(dim=['yh', 'xh']).item()
        current_max=data_var.isel(time=t).max(dim=['yh', 'xh']).item()
        min_val=min(min_val,current_min)
        max_val=max(max_val,current_max)

    #RETURNING
    if var in ['qv']:
        min_val*=1000
        max_val*=1000
    return (min_val,max_val)

def minmax_surface_val_3D(data, var):
    #Data must be dimension (Time,Z,Y,X)
    print(f'working on consistent contour value for {var}')
    data_var = data[var]
    
    t=0
    min_val=data_var.isel(time=t,zh=0).min(dim=['yh', 'xh']).item()
    max_val=data_var.isel(time=t,zh=0).max(dim=['yh', 'xh']).item()
    
    for t in np.arange(1,len(ModelData.time)):
        if np.mod(t,50)==0: print(f'current time {t}')
    
        current_min=data_var.isel(time=t,zh=0).min(dim=['yh', 'xh']).item()
        current_max=data_var.isel(time=t,zh=0).max(dim=['yh', 'xh']).item()
        min_val=min(min_val,current_min)
        max_val=max(max_val,current_max)

    #RETURNING
    if var in ['qv']:
        min_val*=1000
        max_val*=1000
    return (min_val,max_val)

######################################################
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Variable_Calculation/'
def get_theta_v(t): #*#*
    theta_v_data = CallVariable(ModelData, DataManager, ModelData.timeStrings[t], 'theta_v')
    return theta_v_data

def minmax_theta_v():
    # if len(array.shape) != 3:
        # raise ValueError("Data is not 2D + Time")
    print(f'working on theta_v\n')
    
    t=0
    array=get_theta_v(t)
    min_val=np.min(array[0])
    max_val=np.max(array[0])

    for t in np.arange(1,len(ModelData.time)):
        if np.mod(t,50)==0: print(f'current time {t}')
        array=get_theta_v(t)
        current_min=np.min(array[0])
        current_max=np.max(array[0])

        min_val=min(min_val,current_min)
        max_val=max(max_val,current_max)
    
    return (min_val,max_val)

def get_MSE(t): #*#*
    Cpd=1005.7
    MSE_data = CallVariable(ModelData, DataManager, ModelData.timeStrings[t], 'MSE')
    return MSE_data/Cpd

def minmax_MSE():
    # if len(array.shape) != 3:
        # raise ValueError("Data is not 2D + Time")
    print(f'working on mse\n')
    
    t=0
    array=get_MSE(t)
    min_val=np.min(array[0])
    max_val=np.max(array[0])

    for t in np.arange(1,len(ModelData.time)):
        if np.mod(t,50)==0: print(f'current time {t}')
        array=get_MSE(t)
        current_min=np.min(array[0])
        current_max=np.max(array[0])

        min_val=min(min_val,current_min)
        max_val=max(max_val,current_max)
    
    return (min_val,max_val)

In [28]:
def load_vars(data,t):
    surface_qv_data=data['qv'].data[0]*1000
    surface_th_data=data['th'].data[0]
    surface_qvflux_data=data['qvflux'].data
    surface_thflux_data=data['thflux'].data

    import h5py
    theta_v_data=get_theta_v(t)[0]
    surface_mse_data=get_MSE(t)[0]

    return surface_qv_data,surface_th_data,theta_v_data,surface_qvflux_data,surface_thflux_data,surface_mse_data

In [29]:
#RUNNING ANIMATION
###############################################################

In [30]:
#Load Data
data = ModelData.OpenData()

# Store vmin and vmax for each variable in a list of tuples
vmin_max_values = [
    minmax_surface_val_3D(data,'qv'),  
    minmax_surface_val_3D(data,'th'),  
    minmax_theta_v(), 
    minmax_surface_val_2D(data,'qvflux'),  
    minmax_surface_val_2D(data,'thflux'),  
    minmax_MSE()
]

Opened dataset: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
working on consistent contour value for qv
current time 50
current time 100
working on consistent contour value for th
current time 50
current time 100
working on theta_v

current time 50
current time 100
working on consistent contour value for qvflux
current time 50
current time 100
working on consistent contour value for thflux
current time 50
current time 100
working on mse

current time 50
current time 100


In [31]:
#PLOTTING FUNCTION
def MakeSinglePlot(fig, data, t, vmin_max_values):
    gs = gridspec.GridSpec(3, 2, figure=fig, hspace=0.1, wspace=0.1)
    ax1 = fig.add_subplot(gs[0, 0]) #qv
    ax2 = fig.add_subplot(gs[0, 1]) #qv_flux
    ax3 = fig.add_subplot(gs[1,0]) #th
    ax4 = fig.add_subplot(gs[1, 1]) #th_flux
    ax5 = fig.add_subplot(gs[2,0]) #th_v
    ax6 = fig.add_subplot(gs[2,1]) #MSE

    # Extract vmin and vmax from the tuple list for each variable
    vmin_qv, vmax_qv = vmin_max_values[0]
    vmin_th, vmax_th = vmin_max_values[1]
    vmin_th_v, vmax_th_v = vmin_max_values[2]
    vmin_qvflux, vmax_qvflux = vmin_max_values[3]
    vmin_thflux, vmax_thflux = vmin_max_values[4]
    vmin_mse, vmax_mse = vmin_max_values[5]

    # Define the levels using linspace for each variable to ensure consistent color mapping
    qv_levels = np.linspace(vmin_qv, vmax_qv, 100)
    th_levels = np.linspace(vmin_th, vmax_th, 100)
    th_v_levels = np.linspace(vmin_th_v, vmax_th_v, 100)
    qvflux_levels = np.linspace(vmin_qvflux, vmax_qvflux, 100)
    thflux_levels = np.linspace(vmin_thflux, vmax_thflux, 100)
    mse_levels = np.linspace(vmin_mse, vmax_mse, 100)

    #Get the Variables for Current Timestep
    data_t = data.isel(time=t)
    [surface_qv_data, surface_th_data, surface_th_v_data, surface_qvflux_data, surface_thflux_data, surface_mse_data]=load_vars(data_t,t)

    # Create contour plots with consistent levels for each variable
    title_font=25
    titles=["Surface " + r"$q_v$","Surface " + r"$q_v$" + " Flux","Surface " + r"$\theta$",
            "Surface " + r"$\theta$" + " Flux","Surface " + r"$\theta_v$","Surface " + "MSE/Cpd"]
        
    cf1 = ax1.contourf(surface_qv_data, levels=qv_levels)
    ax1.set_title(titles[0] + f" -- Time {get_time(t)[1]}",fontsize=title_font)

    cf2 = ax2.contourf(surface_qvflux_data, levels=qvflux_levels)
    ax2.set_title(titles[1],fontsize=title_font)
    
    cf3 = ax3.contourf(surface_th_data, levels=th_levels)
    ax3.set_title(titles[2],fontsize=title_font)

    cf4 = ax4.contourf(surface_thflux_data, levels=thflux_levels)
    ax4.set_title(titles[3],fontsize=title_font)
    
    cf5 = ax5.contourf(surface_th_v_data, levels=th_v_levels)
    ax5.set_title(titles[4],fontsize=title_font)

    cf6 = ax6.contourf(surface_mse_data, levels=mse_levels)
    ax6.set_title(titles[5],fontsize=title_font)

    from mpl_toolkits.axes_grid1 import make_axes_locatable
    def add_colorbar(ax, cf, label):
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.01)  # Adjust size/padding
        cbar = fig.colorbar(cf, cax=cax)
        cbar.set_label(label, fontsize=25)
        cbar.ax.tick_params(labelsize=20)
        cbar.ax.yaxis.get_offset_text().set_fontsize(25)

    labels=[r'$g/kg$',r'$g/g \: m/s$',r'$T$',r'$K \: m/s$',r'$K$',r'$J/kg·K$']
    for (ax,cf,label) in zip([ax1,ax2,ax3,ax4,ax5,ax6],[cf1,cf2,cf3,cf4,cf5,cf6],labels):
        add_colorbar(ax, cf, label)

    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        tick_font=20
        ax.tick_params(axis='both', which='major', labelsize=tick_font)

In [ ]:
# #TESTING SINGLE_PLOT FUNCTION
# t=50
# # fig = plt.figure(figsize=(35, 15))
# fig = plt.figure(figsize=(60, 25))
# MakeSinglePlot(fig,data,t,vmin_max_values)

# # #TESTING PLOTTING
# # plotFileName = f'QV_TH_animation_{ModelData.res}_{ModelData.Np_str}.jpg'
# # plottingFileName = GetPlottingDirectory(plotFileName=plotFileName, plotType="Initial_Figures/Animations")
# # plt.savefig(plottingFileName, format='jpg')

In [114]:
#ANIMATION FUNCTION
#MUST CREATE A SINGLE TIME PLOTTING FUNCTION TITLED single_plot(args) 

from matplotlib.animation import FuncAnimation, PillowWriter
def create_animation(start_t, end_t, plottingFileName, vmin_max_values, fps=2):
    # Create a figure each time for the animation
    fig = plt.figure(figsize=(60, 25))

    # Define the update function for the animation
    def update(t):
        plt.clf()  # Clear the current figure
        if np.mod(t, 20) == 0:
            print(f'current t: {t}')
        MakeSinglePlot(fig, data, t, vmin_max_values)  # Pass the figure and vmin_max_values to single_plot

    # Create the animation object
    ani = FuncAnimation(fig, update, frames=np.arange(start_t, end_t), repeat=False)

    # Save the animation as a GIF file using PillowWriter
    writer = PillowWriter(fps=fps) 
    ani.save(plottingFileName, writer=writer)
    plt.close(fig)


# # RUNNING CODE
# output_filename = dir+'animations/QV_TH_animation_1km.gif'
# create_animation(start_t=33, end_t=35, output_file=output_filename, vmin_max_values=vmin_max_values)
# # create_animation(start_t=0, end_t=len(netCDF['time'])-1, output_file=output_filename, vmin_max_values=vmin_max_values)


plotFileName = f'QV_TH_animation_{ModelData.res}_{ModelData.Np_str}.gif'
plottingFileName = GetPlottingDirectory(plotFileName=plotFileName, plotType="Initial_Figures/Animations")

create_animation(start_t=0, end_t=len(ModelData.time)-1, plottingFileName=plottingFileName, vmin_max_values=vmin_max_values)

# #TESTING
# create_animation(start_t=0, end_t=3, plottingFileName=plottingFileName, vmin_max_values=vmin_max_values) 

current t: 0
current t: 0
current t: 20
current t: 40
current t: 60
current t: 80
current t: 100
current t: 120


In [115]:
def convert_gif_to_mp4(input_file, output_file, fps,speed,bitrate='750k'):
    from moviepy.editor import VideoFileClip, vfx
    # Load the GIF file
    gif_clip = VideoFileClip(input_file)

    # Set the desired framerate if provided
    if fps:
        gif_clip = gif_clip.set_fps(fps)
    if speed != 1.0:
        gif_clip = gif_clip.fx(vfx.speedx, speed)

    # Write the GIF as an MP4 file
    gif_clip.write_videofile(output_file, codec="libx264",bitrate=bitrate)



#RUNNING CONVERSION
# res='1km'
# input_filename = dir + 'Animations/QV_TH_animation_'+res+'.gif'
# output_filename = dir + 'Animations/QV_TH_animation_'+res+'.mp4'
# convert_gif_to_mp4(input_filename, output_filename, fps=None)  # Optional fps argument

input_name = f'QV_TH_animation_{ModelData.res}_{ModelData.Np_str}.gif'
input_filename = GetPlottingDirectory(plotFileName=input_name, plotType="Initial_Figures/Animations")
output_name = f'QV_TH_animation_{ModelData.res}_{ModelData.Np_str}.mp4'
output_filename = GetPlottingDirectory(plotFileName=output_name, plotType="Initial_Figures/Animations")

if ModelData.res=='1km':
    convert_gif_to_mp4(input_filename, output_filename, fps=None,speed=2,bitrate='750k')  # Optional fps argument
elif ModelData.res=='250m':
    convert_gif_to_mp4(input_filename, output_filename, fps=None,speed=10,bitrate='750k')  # Optional fps argument

Moviepy - Building video /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/PLOTTING/Initial_Figures/Animations/1km_5min_34nz/QV_TH_animation_1km_1e6.mp4.
Moviepy - Writing video /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/PLOTTING/Initial_Figures/Animations/1km_5min_34nz/QV_TH_animation_1km_1e6.mp4



Moviepy - Done !
Moviepy - video ready /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/PLOTTING/Initial_Figures/Animations/1km_5min_34nz/QV_TH_animation_1km_1e6.mp4
